In [117]:
import pandas as pd
import datetime as dt
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import numpy as np
import time
import plotly.graph_objects as go

In [118]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\alpha_vantage_api.txt','r') as f:
    alpha_vantage_api = f.readlines()[0]

In [9]:
#TODO: perform analysis in Jupyter notebook - decide on information to query
#TODO: choose information to store in RDBMS
#TODO: define schema in RDBMS
#TODO: build airflow pipeline to extract, transform and load into RDBMS
#TODO: build dash (dash over streamlit for adaptability)
#TODO: deploy dash
#TODO: have highlighted areas in basketball court that shows top 10 players in every range of shooting based on FG%

># Request for Traditional Endpoint Data

In [119]:
url = "https://stats.nba.com/stats/leaguedashplayershotlocations?" # /leaguedashplayerstats?, /playerestimatedmetrics?, /leaguedashplayerclutch?, /leagueleaders? /playergamelogs?, /leaguedashplayershotlocations?

parameters = {
'College': '', 
'Conference': '', # East, West
'Country': '',
'DateFrom': '',
'DateTo': '',
'DistanceRange': '5ft Range', # (Only for leaguedashplayershotlocations) 5ft Range, 8ft Range, By Zone
'Division': '', # Central, Atlantic, Southeast, Southwest, Northwest, Pacific 
'DraftPick': '', # 1st Round, 2nd Round, 1st Pick, Lottery Pick, Top 5 Pick, Top 10 Pick, Top 15 Pick, Top 20 Pick, Top 25 Pick, Picks 11 Thru 20, Picks 21 Thru 30, Undrafted
'DraftYear': '',
'GameScope': '',
'GameSegment': '', # First Half, Second Half, Overtime
'Height': '', # LT 6-0, GT 6-9
'LastNGames': '0',
'LeagueID': '00',
'Location': '', # Home, Road
'MeasureType': 'Base', # Base, Advanced, Misc, Scoring, Usage, Opponent, Defense, 
'Month': '0', # January=4, February=5, ..., September=9, October=1, November=2, December=3
'OpponentTeamID': '0',
'Outcome': '', # W, L
'PORound': '0', #Conference Quarter-Finals=1, Conference Semi-Finals=2, Conference Finals=3, Finals=4
'PaceAdjust': 'N',
'PerMode': 'Totals', #Per Mode: Totals, PerGame, Per100Possessions, Per100Plays, Per48, Per40, Per36, PerMinute, PerPossession, PerPlay, MinutesPer
'Period': '0', #Quarter: 1,2,3,4,5,6,7,8,9,10
'PlayerExperience': '', # Rookie, Sophomore, Veteran
'PlayerPosition': '', # Center=C, Forward=F, Guard=G
'PlusMinus': 'N',
'Rank': 'N',
'Season': '2022-23', # YYYY-YY, All Time for leagueleaders endpoint
'SeasonSegment': '', # Pre All-Star, Post All-Star
'SeasonType': 'Regular Season', # Season Type: Regular Season, Playoffs, Pre Season, All Star, PlayIn
'StatCategory': '', # (Only for leagueleaders endpoint) PTS, MIN, FGA, FGM, TOV, FG_PCT, FG3_PCT, FG3A, FG3M, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, BLK, STL, 
'ShotClockRange': '', # 24-22, 22-18 Very Early, 18-15 Very Early, 15-7 Average, 7-4 Late, 4-0 Very Late
'StarterBench': '', # Starters, Bench
'TeamID': '0',
'VsConference': '', # East, West
'VsDivision': '', # Central, Atlantic, Southeast, Southwest, Northwest, Pacific 
'Weight': '', # GT 200, LT 200, 
'AheadBehind': '', # (Only for clutch endpoint) Ahead or Behind, Behind or Tied, Ahead or Tied
'ActiveFlag': '', # (Only for leagueleaders endpoint) No, Yes
'ClutchTime': '', # (Only for clutch endpoint) Last 5 mins, Last 4 mins, Last 3 mins, Last 2 mins, Last 1 mins, Last 30 Seconds, Last 10 Seconds 
'PointDiff': '', # (Only for clutch endpoint) 5, 4, 3, 2, 1
}

headers = {
'Accept': '*/*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9',
'Connection': 'keep-alive',
'Host': 'stats.nba.com',
'Origin': 'https://www.nba.com',
'Referer': 'https://www.nba.com/',
'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "Windows",
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

r = requests.request('GET', url=url, headers=headers, params=parameters)

In [4]:
r = r.json()

In [78]:
r['resultSets']['headers']

[{'name': 'SHOT_CATEGORY',
  'columnsToSkip': 6,
  'columnSpan': 3,
  'columnNames': ['Less Than 5 ft.',
   '5-9 ft.',
   '10-14 ft.',
   '15-19 ft.',
   '20-24 ft.',
   '25-29 ft.',
   '30-34 ft.',
   '35-39 ft.',
   '40+ ft.']},
 {'name': 'columns',
  'columnSpan': 1,
  'columnNames': ['PLAYER_ID',
   'PLAYER_NAME',
   'TEAM_ID',
   'TEAM_ABBREVIATION',
   'AGE',
   'NICKNAME',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT',
   'FGM',
   'FGA',
   'FG_PCT']}]

In [5]:
n_group = 3
to_be_grouped = r['resultSets']['headers'][1]['columnNames'][6:]
grouped = []

for i in range(0, len(to_be_grouped), n_group):
    mini_groups = to_be_grouped[i:i+n_group]
    grouped.append(mini_groups)

grouped

[['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT'],
 ['FGM', 'FGA', 'FG_PCT']]

In [6]:
shotDistance = r['resultSets']['headers'][0]['columnNames']
stats_category = grouped[0]

In [7]:
new_column_index = [shotDistance, stats_category]
index = pd.MultiIndex.from_product(new_column_index, names=['shotDistance','category'])

In [120]:
index = [
    ("", 'PLAYER_ID'),
    ("", 'PLAYER_NAME'),
    ("", 'TEAM_ID'),
    ("", 'TEAM_ABBREVIATION'),
    ("", 'AGE'),
    ("", 'NICKNAME'),
    ('Less Than 5 ft.',    'FGM'),
    ('Less Than 5 ft.',    'FGA'),
    ('Less Than 5 ft.', 'FG_PCT'),
    (        '5-9 ft.',    'FGM'),
    (        '5-9 ft.',    'FGA'),
    (        '5-9 ft.', 'FG_PCT'),
    (      '10-14 ft.',    'FGM'),
    (      '10-14 ft.',    'FGA'),
    (      '10-14 ft.', 'FG_PCT'),
    (      '15-19 ft.',    'FGM'),
    (      '15-19 ft.',    'FGA'),
    (      '15-19 ft.', 'FG_PCT'),
    (      '20-24 ft.',    'FGM'),
    (      '20-24 ft.',    'FGA'),
    (      '20-24 ft.', 'FG_PCT'),
    (      '25-29 ft.',    'FGM'),
    (      '25-29 ft.',    'FGA'),
    (      '25-29 ft.', 'FG_PCT'),
    (      '30-34 ft.',    'FGM'),
    (      '30-34 ft.',    'FGA'),
    (      '30-34 ft.', 'FG_PCT'),
    (      '35-39 ft.',    'FGM'),
    (      '35-39 ft.',    'FGA'),
    (      '35-39 ft.', 'FG_PCT'),
    (        '40+ ft.',    'FGM'),
    (        '40+ ft.',    'FGA'),
    (        '40+ ft.', 'FG_PCT')
]

In [121]:
df = pd.DataFrame.from_records(r['resultSets']['rowSet'], columns=pd.MultiIndex.from_tuples(index))
df

TypeError: 'Response' object is not subscriptable

In [122]:
for i in df.columns.to_flat_index():
    print(" ".join(i).strip())

PLAYER_ID
PLAYER_NAME
TEAM_ID
TEAM_ABBREVIATION
AGE
NICKNAME
Less Than 5 ft. FGM
Less Than 5 ft. FGA
Less Than 5 ft. FG_PCT
5-9 ft. FGM
5-9 ft. FGA
5-9 ft. FG_PCT
10-14 ft. FGM
10-14 ft. FGA
10-14 ft. FG_PCT
15-19 ft. FGM
15-19 ft. FGA
15-19 ft. FG_PCT
20-24 ft. FGM
20-24 ft. FGA
20-24 ft. FG_PCT
25-29 ft. FGM
25-29 ft. FGA
25-29 ft. FG_PCT
30-34 ft. FGM
30-34 ft. FGA
30-34 ft. FG_PCT
35-39 ft. FGM
35-39 ft. FGA
35-39 ft. FG_PCT
40+ ft. FGM
40+ ft. FGA
40+ ft. FG_PCT


# Scrape

### Define years to scrape

In [123]:
start_year = 1996
end_year = 2022
years = []

while start_year <= end_year:
    years.append(f'{start_year}-{str(start_year+1)[-2:]}')
    start_year += 1

### Time between requests

In [124]:
lag = np.random.randint(low=5, high=25)

### Function for Scraping

In [125]:
def scrape_nba(year, **kwargs):
    url = "https://stats.nba.com/stats/leaguedashplayershotlocations?" # /leaguedashplayerstats?, /playerestimatedmetrics?, /leaguedashplayerclutch?, /leagueleaders? /playergamelogs?, /leaguedashplayershotlocations?
    
    parameters = {
    # 'AheadBehind': '', # (Only for clutch endpoint) Ahead or Behind, Behind or Tied, Ahead or Tied
    # 'ActiveFlag': '', # (Only for leagueleaders endpoint) No, Yes
    # 'ClutchTime': '', # (Only for clutch endpoint) Last 5 mins, Last 4 mins, Last 3 mins, Last 2 mins, Last 1 mins, Last 30 Seconds, Last 10 Seconds 
    'College': '', 
    'Conference': '', # East, West
    'Country': '',
    'DateFrom': '',
    'DateTo': '',
    'DistanceRange': '5ft Range', # (Only for leaguedashplayershotlocations) 5ft Range, 8ft Range, By Zone
    'Division': '', # Central, Atlantic, Southeast, Southwest, Northwest, Pacific 
    'DraftPick': '', # 1st Round, 2nd Round, 1st Pick, Lottery Pick, Top 5 Pick, Top 10 Pick, Top 15 Pick, Top 20 Pick, Top 25 Pick, Picks 11 Thru 20, Picks 21 Thru 30, Undrafted
    'DraftYear': '',
    'GameScope': '',
    'GameSegment': '', # First Half, Second Half, Overtime
    'Height': '', # LT 6-0, GT 6-9
    'LastNGames': '0',
    'LeagueID': '00',
    'Location': '', # Home, Road
    'MeasureType': 'Base', # Base, Advanced, Misc, Scoring, Usage, Opponent, Defense, 
    'Month': '0', # January=4, February=5, ..., September=9, October=1, November=2, December=3
    'OpponentTeamID': '0',
    'Outcome': '', # W, L
    'PORound': '0', #Conference Quarter-Finals=1, Conference Semi-Finals=2, Conference Finals=3, Finals=4
    'PaceAdjust': 'N',
    'PerMode': 'Totals', #Per Mode: Totals, PerGame, Per100Possessions, Per100Plays, Per48, Per40, Per36, PerMinute, PerPossession, PerPlay, MinutesPer
    'Period': '0', #Quarter: 1,2,3,4,5,6,7,8,9,10
    'PlayerExperience': '', # Rookie, Sophomore, Veteran
    'PlayerPosition': '', # Center=C, Forward=F, Guard=G
    'PlusMinus': 'N',
    # 'PointDiff': '', # (Only for clutch endpoint) 5, 4, 3, 2, 1
    'Rank': 'N',
    'Season': year, # YYYY-YY, All Time for leagueleaders endpoint
    'SeasonSegment': '', # Pre All-Star, Post All-Star
    'SeasonType': 'Regular Season', # Season Type: Regular Season, Playoffs, Pre Season, All Star, PlayIn
    'StatCategory': '', # (Only for leagueleaders endpoint) PTS, MIN, FGA, FGM, TOV, FG_PCT, FG3_PCT, FG3A, FG3M, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, BLK, STL, 
    'ShotClockRange': '', # 24-22, 22-18 Very Early, 18-15 Very Early, 15-7 Average, 7-4 Late, 4-0 Very Late
    'StarterBench': '', # Starters, Bench
    'TeamID': '0',
    'VsConference': '', # East, West
    'VsDivision': '', # Central, Atlantic, Southeast, Southwest, Northwest, Pacific 
    'Weight': '', # GT 200, LT 200, 
    }

    headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Host': 'stats.nba.com',
    'Origin': 'https://www.nba.com',
    'Referer': 'https://www.nba.com/',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': "Windows",
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }

    r = requests.request('GET', url=url, headers=headers, params=parameters)
    r = r.json()
    return r

In [126]:
def process_scraped_data(result, index):
    df = pd.DataFrame.from_records(result['resultSets']['rowSet'], columns=pd.MultiIndex.from_tuples(index))
    return df

### Scrape

In [127]:
combined_df = []
begin_time = time.time()
lag_time = 0
for year in years:
    r = scrape_nba(year)
    df = process_scraped_data(result=r, index=index)
    df['year'] = year
    combined_df.append(df)
    print(f'Successfully retrieved data for Season {year}')
    lag = np.random.randint(low=5, high=25)
    lag_time += lag
    print(f'Delaying for: {lag}s\nCurrent total lag time: {lag_time}s')
    time.sleep(lag)
time_taken = time.time() - begin_time
print(f'Total time taken: {round(time_taken/60, 2)}mins')

Successfully retrieved data for Season 1996-97
Delaying for: 12s
Current total lag time: 12s
Successfully retrieved data for Season 1997-98
Delaying for: 7s
Current total lag time: 19s
Successfully retrieved data for Season 1998-99
Delaying for: 19s
Current total lag time: 38s
Successfully retrieved data for Season 1999-00
Delaying for: 21s
Current total lag time: 59s
Successfully retrieved data for Season 2000-01
Delaying for: 20s
Current total lag time: 79s
Successfully retrieved data for Season 2001-02
Delaying for: 15s
Current total lag time: 94s
Successfully retrieved data for Season 2002-03
Delaying for: 24s
Current total lag time: 118s
Successfully retrieved data for Season 2003-04
Delaying for: 19s
Current total lag time: 137s
Successfully retrieved data for Season 2004-05
Delaying for: 23s
Current total lag time: 160s
Successfully retrieved data for Season 2005-06
Delaying for: 15s
Current total lag time: 175s
Successfully retrieved data for Season 2006-07
Delaying for: 23s
Cu

In [131]:
final_df = pd.concat(combined_df, axis=0)

In [132]:
final_df.to_csv('player_shotLocations.csv', index=False)

In [133]:
player_ID = final_df[[("", 'PLAYER_ID'),("", 'PLAYER_NAME'), ("", 'NICKNAME')]].drop_duplicates(subset=("", 'PLAYER_ID'))
team_ID = final_df[[("", 'TEAM_ID'),("","TEAM_ABBREVIATION")]].drop_duplicates()

In [134]:
print(f'''
Number of players from 1996-2023: {len(player_ID)}
Number of teams from 1996-2023: {len(team_ID)}
''')


Number of players from 1996-2023: 2551
Number of teams from 1996-2023: 36



In [79]:
from nba_api import NBA

In [98]:
len([
            {"text": name, "stretch": False} for name in [" ".join(i).strip()
                                                          for i in index]
        ])

33

In [135]:
pd.read_csv('player_shotLocations.csv', header=1, index_col=0)

,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,NICKNAME,FGM,FGA,FG_PCT,FGM.1,FGA.1,...,FGM.6,FGA.6,FG_PCT.6,FGM.7,FGA.7,FG_PCT.7,FGM.8,FGA.8,FG_PCT.8,Unnamed: 33
PLAYER_ID,,,,,,,,,,,,,,,,,,,,,
920,A.C. Green,1610612742,DAL,33.0,A.C.,136.0,235.0,0.579,11.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.167,1996-97
243,Aaron McKie,1610612765,DET,24.0,Aaron,72.0,129.0,0.558,4.0,32.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.000,1996-97
1425,Aaron Williams,1610612763,VAN,25.0,Aaron,76.0,114.0,0.667,7.0,23.0,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000,1996-97
768,Acie Earl,1610612749,MIL,27.0,Acie,31.0,66.0,0.470,26.0,53.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,1996-97
228,Adam Keefe,1610612762,UTA,27.0,Adam,61.0,107.0,0.570,6.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628380,Zach Collins,1610612759,SAS,25.0,Zach,154.0,234.0,0.658,46.0,94.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,2022-23
203897,Zach LaVine,1610612741,CHI,28.0,Zach,309.0,469.0,0.659,33.0,81.0,...,4.0,8.0,0.5,0.0,3.0,0.0,0.0,0.0,0.000,2022-23
1630192,Zeke Nnaji,1610612743,DEN,22.0,Zeke,90.0,124.0,0.726,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.000,2022-23


In [83]:
[tuple(row) for row in df.itertuples()]

[(0,
  1630639,
  'A.J. Lawson',
  1610612742,
  'DAL',
  22.0,
  'A.J.',
  11,
  18,
  0.611,
  1.0,
  1.0,
  1.0,
  0,
  0,
  0.0,
  0.0,
  0.0,
  0.0,
  4.0,
  17.0,
  0.235,
  6.0,
  8.0,
  0.75,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0),
 (1,
  1631260,
  'AJ Green',
  1610612749,
  'MIL',
  23.0,
  'AJ',
  3,
  6,
  0.5,
  0.0,
  1.0,
  0.0,
  1,
  3,
  0.333,
  4.0,
  8.0,
  0.5,
  12.0,
  30.0,
  0.4,
  31.0,
  74.0,
  0.419,
  2.0,
  3.0,
  0.667,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0),
 (2,
  1631100,
  'AJ Griffin',
  1610612737,
  'ATL',
  19.0,
  'AJ',
  50,
  80,
  0.625,
  33.0,
  62.0,
  0.532,
  48,
  88,
  0.545,
  16.0,
  37.0,
  0.432,
  47.0,
  116.0,
  0.405,
  54.0,
  149.0,
  0.362,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (3,
  203932,
  'Aaron Gordon',
  1610612743,
  'DEN',
  27.0,
  'Aaron',
  326,
  456,
  0.715,
  19.0,
  55.0,
  0.345,
  14,
  42,
  0.333,
  9.0,
  34.0,
  0.265,
  22.0,
  72.0,
  0.306,
  3